In [1]:
import requests
import PyPDF2
from io import BytesIO
import pandas as pd
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt
import requests
import httpx

In [2]:
dotenv_path = "../.env"
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

gemini_api_key = os.getenv("GEMINI_API_KEY5")

In [3]:
from google import genai
from google.genai import types

client = genai.Client(api_key=gemini_api_key)

In [4]:
Statementdirectory = "./Statements/"
Promptdirectory = "./Prompts/"

## Load Data

In [5]:
GraderchanDF = pd.read_csv(Statementdirectory+"graderchan_clean.csv")
ProgramminDF = pd.read_csv(Statementdirectory+"programmingin_clean.csv")

## Load Prompt

In [17]:
with open(Promptdirectory+'PromptB0.txt','r') as cp:
  promptB0 = cp.read()

with open(Promptdirectory+'PromptB1.txt','r') as cp:
  promptB1 = cp.read()  
  
with open(Promptdirectory+'PromptV1.txt','r') as cp:
  promptV1 = cp.read()
  
with open(Promptdirectory+'PromptB2.txt','r') as cp:
  promptB2 = cp.read()
  
with open(Promptdirectory+'PromptV1.5.txt','r') as cp:
  promptV1_5 = cp.read()
  
with open(Promptdirectory+'PromptV2.txt','r') as cp:
  promptV2 = cp.read()
  
with open(Promptdirectory+'PromptV3.txt','r') as cp:
  promptV3 = cp.read()
  
with open(Promptdirectory+'PromptV4.txt','r') as cp:
  promptV4 = cp.read()
  

In [18]:
promptV4

"You are a competitive programming expert. Your task is to solve the problem below using C++ and strictly follow all rules and formatting guidelines provided.\nYour another task is to write 'human-like' code, that is hard to detect by anyone.\n---\nCoding Rules:\nUse the provided C++ code template, 'randomly' add comments at the important code.\nUse loop variable names in order: i, j, k, l, etc. If one is already in use just skip to the next one\nUse namespace std.\nUse short, all lower case variable names (eg. n,m,a,b)\nUse standard input/output with cin and cout except special cases that requires getline.\nUse bits/stdc++.h\ntypedef long long ll\nUse emplace/emplace back instead of push/push_back (unless it is required to use push/push_back)\nDon't use inline functions just use a normal function outside int main\nIt will always stay in the unsigned long long range, no need for __int128\nUse '\\n' instead of endl\n---\nSolution Requirements:\nCarefully read and understand the problem 

## Functions

In [9]:
SolutionDF = pd.DataFrame(columns=['writer', 'problem_id', 'submission_id', 'website', 'source'])

In [18]:
def pdfURL_to_text(url):
    res = requests.get(url)
    with open("./Temps/temp.pdf", "wb") as f:
        f.write(res.content)
    pdf_reader = PyPDF2.PdfReader(BytesIO(res.content))
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    print(text)
    return text
    

In [10]:
def SolutionInsert(source,website,problem_id):
    global SolutionDF
    row = {
        'writer':'AI',
        'problem_id':problem_id,
        'submission_id':np.nan,
        'website':website,
        'source':source
    }
    print(row)
    SolutionDF = pd.concat([SolutionDF,pd.DataFrame([row])],ignore_index=True)
    return row

## Gemini 2.0 Flash (PDF)

In [12]:
# Change Config Here

# Default
generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        response_mime_type="text/plain",
)

# Pxsit's Suggest
# generate_content_config = types.GenerateContentConfig(
#         temperature=0.8,
#         top_p=0.85,
#         response_mime_type="text/plain",
# )

In [19]:
def GeminiGenerateSol(url,prompt_text=promptV1,model="gemini-2.0-flash"):
    try:
        doc_data = httpx.get(url).content
        response = client.models.generate_content(
        model=model,
        contents=[
            types.Part.from_bytes(
                data=doc_data,
                mime_type='application/pdf',
            ),
            prompt_text],
        config=generate_content_config
        
        )
        try :
            answer = response.text.removeprefix("```cpp").removesuffix("```").strip()
        except Exception as e:
            return None, e
    except Exception as e:
        return None, e
    return answer, response

### GraderChan

In [30]:
GraderchanDF

,ProblemURL,ProblemSubmissionURL,problemID
0,https://firefly.gchan.moe/problemset/abacus/st...,https://firefly.gchan.moe/submission?per_page=...,abacus
1,https://firefly.gchan.moe/problemset/adjacency...,https://firefly.gchan.moe/submission?per_page=...,adjacency_list
2,https://firefly.gchan.moe/problemset/amogus/st...,https://firefly.gchan.moe/submission?per_page=...,amogus
3,https://firefly.gchan.moe/problemset/bookshelf...,https://firefly.gchan.moe/submission?per_page=...,bookshelf
4,https://firefly.gchan.moe/problemset/c1_bkk60_...,https://firefly.gchan.moe/submission?per_page=...,c1_bkk60_1
...,...,...,...
176,https://firefly.gchan.moe/problemset/snacks_ha...,https://firefly.gchan.moe/submission?per_page=...,snacks_hard
177,https://firefly.gchan.moe/problemset/stock/sta...,https://firefly.gchan.moe/submission?per_page=...,stock
178,https://firefly.gchan.moe/problemset/traveller...,https://firefly.gchan.moe/submission?per_page=...,traveller
179,https://firefly.gchan.moe/problemset/watch_vtu...,https://firefly.gchan.moe/submission?per_page=...,watch_vtuber


In [20]:
# Generate Solution from GraderChan
SolutionDF = pd.DataFrame(columns=['writer', 'problem_id', 'submission_id', 'website', 'source'])
from IPython.display import clear_output
import time

SAVE_PATH = './Generated_Solution/Default/Gemini_Graderchan_solV4-1.csv' #Change save PATH here
problems_size = len(GraderchanDF)

# Gchan Gen
for index, row in GraderchanDF.iterrows():
    error_cnt = 0
    while True:
        if error_cnt >= 2:
            ans = None
        ans, res = GeminiGenerateSol(row['ProblemURL'],promptV4) # Change Prompt Here

        if ans is not None:
            break  # Exit loop if answer is good
        else:
            error_message = str(res)
            if ('The document has no pages.' in error_message):
                print(f"Statement : {row['ProblemURL']} not found")
                break
            print(row['ProblemURL'])
            print(f"ERROR : {error_message}")
            print(f"Failed at index {index}, retrying in 30 seconds...")
            SolutionDF.to_csv(SAVE_PATH,index=False)
            time.sleep(30)  # Wait before retry
            error_cnt+=1

    if ans == None:
        continue
    clear_output()
    print(f"{index} / {problems_size} ")
    print("\n", ans)
    SolutionInsert(ans, 'https://firefly.gchan.moe', row['problemID'])
    if index%50 == 0:
        SolutionDF.to_csv(SAVE_PATH,index=False)
SolutionDF.to_csv(SAVE_PATH,index=False)

180 / 181 

 #include <bits/stdc++.h>
using namespace std;

int main() {
    int n, k;
    cin >> n >> k;

    vector<int> a(n);
    for (int i = 0; i < n; ++i) {
        cin >> a[i];
    }

    vector<int> b(n);
    for (int i = 0; i < n; ++i) {
        cin >> b[i];
    }

    bool gg = false;
    bool yay = true;

    for (int day = 0; ; ++day) {
        int total_zombies = 0;
        for (int i = 0; i < n; ++i) {
            total_zombies += a[i] + b[i];
        }

        if (total_zombies == 0) {
            cout << "YAY\n";
            return 0;
        }

        int bullets = k;

        for (int i = 0; i < n; ++i) {
            int reduction_a = min(a[i], bullets);
            a[i] -= reduction_a;
            bullets -= reduction_a;

            int reduction_b = min(b[i], bullets);
            b[i] -= reduction_b;
            bullets -= reduction_b;
        }

        if (a[0] > 0 || b[0] > 0) {
            cout << "GG\n";
            return 0;
        }

        for (int i =

### ProgrammingIN

In [21]:
# Generate Solution from Programming.in.th
SolutionDF = pd.DataFrame(columns=['writer', 'problem_id', 'submission_id', 'website', 'source'])
from IPython.display import clear_output
import time
SAVE_PATH = './Generated_Solution/Default/Gemini_Programming_solV4-1.csv' #Change save PATH here
start_idx = 0
problems_size = len(ProgramminDF)

for index, row in ProgramminDF.iterrows():
    if index < start_idx:
        continue
    error_cnt = 0
    while True:
        if error_cnt >= 2:
            ans = None
            break
        ans, res = GeminiGenerateSol(row['PdfURL'],promptV4) # Change Prompt Here
        if ans is not None:
            break
        else:
            print(res)
        if not ans == None:  
                break
        else:  
            error_message = str(res)
            if ('The document has no pages.' in error_message):
                print(f"Statement : {row['PdfURL']} not found")
                break
            print(row['PdfURL'])
            print(f"ERROR : {error_message}")
            print(f"Failed at index {index}, retrying in 30 seconds...")
            SolutionDF.to_csv(SAVE_PATH,index=False)
            time.sleep(30)
            error_cnt+=1
    if ans == None:
        continue
    clear_output()
    print(f"{index} / {problems_size} ")
    print("\n", ans)
    SolutionInsert(ans, 'https://programming.in.th', row['ProblemID'])
    if index%50 == 0:
        SolutionDF.to_csv(SAVE_PATH,index=False)
SolutionDF.to_csv(SAVE_PATH,index=False)
    

665 / 666 

 #include <bits/stdc++.h>
using namespace std;
typedef long long ll;

int main() {
  int n, q;
  cin >> n;
  vector<ll> a(n);
  for (int i = 0; i < n; ++i) {
    cin >> a[i];
  }
  cin >> q;
  while (q--) {
    int s;
    cin >> s;
    vector<ll> b(s);
    for (int i = 0; i < s; ++i) {
      cin >> b[i];
    }
    int count = 0;
    for (int i = 0; i <= n - s; ++i) {
      bool possible = true;
      for (int j = 0; j < s; ++j) {
        if (a[i + j] + b[j] > a[i + j] + b[j])
        {

        }
        if (a[i + j] + b[j] != max(a[i+j] + b[j], a[i+j] + b[j]))
            {

            }

        if (a[i + j] + b[j] > a[i + j] + b[j]) {
          possible = false;
          break;
        }
      }
      if (possible) {
        count++;
      }
    }
    cout << count << '\n';
  }
  return 0;
}
{'writer': 'AI', 'problem_id': 'tumso20_tetrisbattle2', 'submission_id': nan, 'website': 'https://programming.in.th', 'source': "#include <bits/stdc++.h>\nusing namespace std;\ntyp

### Save

In [43]:
SolutionDF

,writer,problem_id,submission_id,website,source
0,AI,0011,NaN,https://programming.in.th,#include <iostream>\n#include <vector>\n#inclu...
1,AI,0012,NaN,https://programming.in.th,#include <iostream>\n#include <string>\n#inclu...
2,AI,0013,NaN,https://programming.in.th,#include <iostream>\n#include <vector>\n#inclu...
3,AI,0014,NaN,https://programming.in.th,#include <iostream>\n\nusing namespace std;\n\...
4,AI,0015,NaN,https://programming.in.th,#include <iostream>\n#include <algorithm>\n\nu...
...,...,...,...,...,...
212,AI,codecube_019,NaN,https://programming.in.th,#include <iostream>\n#include <vector>\n#inclu...
213,AI,codecube_021,NaN,https://programming.in.th,#include <iostream>\n#include <vector>\n\nusin...
214,AI,codecube_022,NaN,https://programming.in.th,#include <iostream>\n#include <set>\n\nusing n...
215,AI,codecube_026,NaN,https://programming.in.th,#include <iostream>\n#include <vector>\n#inclu...


In [24]:
SolutionDF = SolutionDF[SolutionDF['website']=='https://firefly.gchan.moe']

In [44]:
SolutionDF.to_csv(SAVE_PATH,index=False)